In [1]:
# header files
import glob
import csv
import numpy as np
import pandas as pd
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import (
    concordance_index_censored,
    concordance_index_ipcw,
    cumulative_dynamic_auc,
    integrated_brier_score,
)
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression, f_classif
from datetime import datetime
print("Header files loaded!")

Header files loaded!


In [2]:
# hyper-parameters
date_format = "%m/%d/%Y"
is_ovarian_cancer = 1
is_cervix_cancer = 0
is_endometrial_cancer = 0

def mean(a):
    return sum(a) / len(a)

In [3]:
censor = []
days = []
filenames = []
flag = -1
with open("../../../Desktop/Yale_YTMA79_HEIF_clinical_info_v2.csv", newline='', encoding = "ISO-8859-1") as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        if flag == -1:
            flag = 1
            print(row)
        else:
            array = row
            
            if array[1] == "" or array[22] == "NA":
                continue
            
            filenames.append(str(array[0][1:len(array[0])-1]))
            if array[26] == "1":
                censor.append(False)
                days.append(int(array[22]))
            else:
                censor.append(True)
                days.append(int(array[22]))
print(len(filenames))
print(len(censor))
print(len(days))

['TMA_label', 'Gender', 'Race', 'Diagnosis', 'DiagnosisDate', 'AgeatDiagnosis', 'AliveDeadCensor', 'AliveDeadStatus', 'DateLastSeen', 'DODCensor', 'DODStatus', 'FollowupDays', 'FollowupMonths', 'PrimaryStatus', 'Core', 'Stage', 'histology_subtype', 'smoking_status', 'CD4', 'CD20', 'CD8', 'OS_m', 'OS_d', 'DSS_censor', 'DSS_censor_bin', 'OS_Censor', 'OS_bin']
116
116
116


In [4]:
print(filenames[0])
print(censor[0])
print(days[0])

YTMA79-10_2_383_254
False
1376


In [5]:
print(filenames[50])
print(censor[50])
print(days[50])

YTMA79-10_120_309_943
True
701


In [6]:
ec_files = (glob.glob("../../yale_lung_cancer/macrophage_output/*"))
print(len(ec_files))

119


In [7]:
final_filenames = []
final_censor = []
final_days = []
index = 0
for file in filenames:
    flag = -1
    for file_1 in ec_files:
        if file in file_1:
            final_filenames.append(filenames[index])
            final_censor.append(censor[index])
            final_days.append(days[index])
            break
    index += 1
print(len(final_filenames))
print(len(final_censor))
print(len(final_days))

114
114
114


In [8]:
print(final_filenames)

['YTMA79-10_2_383_254', 'YTMA79-10_3_486_245', 'YTMA79-10_4_614_221', 'YTMA79-10_5_715_219', 'YTMA79-10_7_914_216', 'YTMA79-10_8_1020_205', 'YTMA79-10_9_1122_196', 'YTMA79-10_11_1332_183', 'YTMA79-10_12_1437_184', 'YTMA79-10_13_1550_179', 'YTMA79-10_20_1340_285', 'YTMA79-10_21_1230_301', 'YTMA79-10_22_1126_303', 'YTMA79-10_27_589_329', 'YTMA79-10_30_298_339', 'YTMA79-10_31_289_460', 'YTMA79-10_34_607_429', 'YTMA79-10_35_715_422', 'YTMA79-10_36_819_423', 'YTMA79-10_38_1020_394', 'YTMA79-10_39_1132_402', 'YTMA79-10_41_1343_389', 'YTMA79-10_43_1552_377', 'YTMA79-10_47_1667_470', 'YTMA79-10_50_1351_489', 'YTMA79-10_53_1025_500', 'YTMA79-10_56_711_529', 'YTMA79-10_57_603_534', 'YTMA79-10_58_506_527', 'YTMA79-10_59_402_547', 'YTMA79-10_63_502_644', 'YTMA79-10_65_715_625', 'YTMA79-10_68_1041_596', 'YTMA79-10_72_1456_571', 'YTMA79-10_79_1460_680', 'YTMA79-10_86_720_722', 'YTMA79-10_87_612_729', 'YTMA79-10_89_407_738', 'YTMA79-10_90_301_754', 'YTMA79-10_92_403_848', 'YTMA79-10_94_609_828', 'YTM

In [9]:
# collect features
collagen_features = []
for file in final_filenames:
    file_features = []
        
    for file_1 in ec_files:
        if file in file_1:
            filename = file_1.split("/")[-1]
            flag = -1
            slide_features = []
            
            with open(file_1, newline='') as csvfile:
                spamreader = csv.reader(csvfile)
                for row in spamreader:
                    if flag == -1:
                        array = row
                        for index in range(0, len(array)):
                            slide_features.append(float(array[index]))
            file_features.append(slide_features)
    
    f = [sum(col) / float(len(col)) for col in zip(*file_features)]
    collagen_features.append(f)
print(len(collagen_features))
print(len(collagen_features[0]))

114
3


In [10]:
print(collagen_features)

[[0.176930606285435, 0.10047847889952152, 0.16838488972508592], [0.14777778777777778, 0.13757397449704142, 0.1489796018367347], [0.1623931723931624, 0.2276923176923077, 0.20246914580246914], [0.07744566217391304, 0.09430439842203547, 0.09079755601226994], [0.18166940443535187, 0.1848184918481848, 0.18634260259259258], [0.12477232329690345, 0.15418503202643172, 0.130879355603272], [0.16926771708283314, 0.07764706882352941, 0.15306123448979592], [0.10160056671537926, 0.05263158894736842, 0.10094851948509484], [0.22448980591836734, 0.22344323344322345, 0.23796792443850268], [0.12831859407079646, 0.07582939388625592, 0.11276949590381426], [0.07533113582781456, 0.14960630921259843, 0.07984497124031008], [0.21039183282793866, 0.07619048619047619, 0.20675454047775946], [0.20934580439252337, 0.1705989210707804, 0.1986970784039088], [0.14270153505446623, 0.024390253902439027, 0.14027631180658873], [0.1983273696176822, 0.028735642183908047, 0.17733474242392444], [0.22678572428571428, 0.137254911

In [11]:
y = []
event = []
survival_time = []
for index in range(0, len(final_censor)):
    y.append([final_censor[index], final_days[index]])
    event.append(final_censor[index])
    survival_time.append(final_days[index])
print(len(y))

114


In [12]:
# generate training set for training model
features = []
for index in range(0, len(final_filenames)):
    #features.append(final_til_features[index]+collagen_features[index])
    features.append(collagen_features[index])
    #features.append(final_til_features[index])
print(len(features))
print(len(features[0]))

114
3


In [13]:
# final training information to be used for training model
features = np.array(features)
y = np.array(y)
event = np.array(event)
survival_time = np.array(survival_time)

In [33]:
# main code for training
iter_scores = []
max_score = -1
dt = dtype=[('Status', '?'), ('Survival_in_days', '<f8')]
for iter in range(100):
    model_score = []
    kf = KFold(n_splits=5, shuffle=True)
    for train_index, test_index in kf.split(features):
        # get the training and validation data
        features_train, features_test = features[train_index], features[test_index]
        y_train, y_test = y[train_index], y[test_index]
        event_train, survival_time_train = event[train_index], survival_time[train_index]
        event_test, survival_time_test = event[test_index], survival_time[test_index]
        y_train = np.array([tuple(row) for row in y_train], dtype=dt)
        y_test = np.array([tuple(row) for row in y_test], dtype=dt)
        
        # feature selection
        scaler = MinMaxScaler()
        features_train = scaler.fit_transform(features_train)
        features_test = scaler.transform(features_test)
        #select = SelectKBest(score_func=chi2, k=len(features[0])-6)
        #features_train_selected = select.fit_transform(features_train, survival_time_train)
        #features_test_selected = select.transform(features_test)
        features_train_df = pd.DataFrame(features_train)
        features_test_df = pd.DataFrame(features_test)
        
        # fit model
        estimator = CoxnetSurvivalAnalysis(l1_ratio=0.99, alpha_min_ratio=0.05)
        estimator.fit(features_train_df, y_train)
        
        # score on validation set
        score, _, _, _, _ = concordance_index_censored(event_test, survival_time_test, estimator.predict(features_test_df))
        model_score.append(score)
        if score > max_score:
            max_score = score
    
    if len(model_score) > 0:
        iter_scores.append(np.mean(model_score))
        max_score = max(max(model_score), max_score)
print(np.mean(iter_scores), np.std(iter_scores))
print(max_score)

0.5266694335632429 0.02046837983496862
0.7588652482269503


In [14]:
# model to be used for external validation
features_train = features
y_train = y
event_train, survival_time_train = event, survival_time
dt = dtype=[('Status', '?'), ('Survival_in_days', '<f8')]
y_train = np.array([tuple(row) for row in y_train], dtype=dt)
        
# feature selection
scaler = MinMaxScaler()
features_train = scaler.fit_transform(features_train)
#select = SelectKBest(score_func=chi2, k=len(features[0])-4)
#features_train_selected = select.fit_transform(features_train, survival_time_train)
features_train_df = pd.DataFrame(features_train)
        
# fit model
estimator = CoxnetSurvivalAnalysis(l1_ratio=0.99, alpha_min_ratio=0.05)
estimator.fit(features_train_df, y_train)

CoxnetSurvivalAnalysis(alpha_min_ratio=0.05, l1_ratio=0.99)

In [15]:
# find prognostic features from model trained above
count = 0
for index1 in range(0, len(estimator.coef_)):
    flag = -1
    for index2 in range(0, len(estimator.coef_[index1])):
        if estimator.coef_[index1][index2] > 0:
            flag = 1
            print(index1)
            print(estimator.coef_[index1][index2])
            break
    if flag == 1:
        count += 1
print()
print("Prognostic features count = " + str(count))

0
0.00024777929030766436
1
0.002203506045013648
2
0.03335743986408717

Prognostic features count = 3


In [16]:
oc_files = (glob.glob("../../tam_results/macrophage_tcga_final/*"))
print(len(oc_files))

100


In [17]:
test_tam_features = []
for file in oc_files:
    filename = file.split("/")[-1]
    flag = -1
    file_features = []
    with open(file, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                array = row
                for index in range(0, len(array)):
                    file_features.append(float(array[index]))
    test_tam_features.append(file_features)
print(test_tam_features)

[[0.00014818997940298286, 0.00014669133480014668, 0.00013132079689896423], [0.007466339966329966, 0.0015420746317838796, 0.004648486179997799], [0.0014688268049782654, 0.00011877345823614847, 0.000730889324187524], [0.000218272857405429, 0.00020376559958776403, 0.00020706643544566802], [0.00596311770040699, 0.0007054363565891474, 0.002128200867507349], [0.00756324974340172, 0.000923806872576719, 0.006664269185749724], [0.014865759836280287, 0.002341181958962025, 0.0053560225003854225], [0.0019514339679498345, 0.0006276857949906241, 0.0012758293428917664], [0.00012741708383386116, 2.2017090684618587e-05, 4.3026380279140464e-05], [0.00221025585200436, 0.0008354712098890367, 0.0015027562377009224], [0.02356725269701613, 0.013774431340199471, 0.02107045556738179], [0.003871532263121508, 0.00051814782466136, 0.0016938873887249795], [0.0013968569952906907, 0.0014748896392865797, 0.0014108510687536049], [0.017570479904256912, 0.009097597785922215, 0.014103201597838278], [0.00913451494726603, 

In [18]:
# Sepideh OC Spatil features
test_censor = []
test_days = []
test_filenames = []
flag = -1
with open("../../DATA_OC.csv", newline='', encoding = "ISO-8859-1") as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        if flag == -1:
            flag = 1
            print(row)
        else:
            array = row
            #if int(array[23]) > 1800:
            #    array[20] = 0
                
            test_filenames.append(array[0])
            test_censor.append(bool(int(array[20])))
            test_days.append(int(array[23]))

print(len(test_filenames))
print(len(test_censor))
print(len(test_days))

['patient_name', 'SF1', 'SF2', 'SF3', 'SF4', 'SF5', 'SF6', 'SF7', 'cont_risk_score', 'binary_risk_score', 'WSI_Width', 'WSI_Height', 'year_of_birth', 'race', 'year_of_death', 'vital_status', 'Organ', 'treatment_type', 'Age', 'TTE', 'censor', 'Site', 'stage', 'OS_days', 'Vital', 'stage_numeric']
103
103
103


In [20]:
test_y = []
test_event = []
test_survival_time = []
for file in oc_files:
    count = 0
    filename1 = file.split("/")[-1][:-4]
    for filename in test_filenames:
        filename2 = filename
        if filename1 == filename2:
            test_y.append([test_censor[count], test_days[count]])
            test_event.append(test_censor[count])
            test_survival_time.append(test_days[count])
        count += 1
print(len(test_y))
print(len(test_event))
print(len(test_survival_time))

100
100
100


In [21]:
# generate training set for training model
test_features = []
for index in range(0, len(oc_files)):
    #features.append(final_til_features[index]+collagen_features[index])
    test_features.append(test_tam_features[index])
    #features.append(final_til_features[index])
print(len(test_features))
print(len(test_features[0]))

100
3


In [22]:
# final training information to be used for training model
test_features = np.array(test_features)
test_y = np.array(test_y)
test_event = np.array(test_event)
test_survival_time = np.array(test_survival_time)

In [31]:
# run on test set
group = []
features_train = test_features
features_test = features
y_train = test_y
event_train, survival_time_train = test_event, test_survival_time
dt = dtype=[('Status', '?'), ('Survival_in_days', '<f8')]
y_train = np.array([tuple(row) for row in y_train], dtype=dt)
scaler = MinMaxScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)
#select = SelectKBest(score_func=chi2, k=len(features[0])-4)
#features_train_selected = select.fit_transform(features_train, survival_time_train)
#features_test_selected = select.transform(features_test)
features_train_df = pd.DataFrame(features_train)
features_test_df = pd.DataFrame(features_test)
        
# fit model
estimator = CoxnetSurvivalAnalysis(l1_ratio=0.99, alpha_min_ratio=0.05)
estimator.fit(features_train_df, y_train)

score, _, _, _, _ = concordance_index_censored(event, survival_time, estimator.predict(features_test_df))
print(score)

# get risk scores
train_risk_scores = estimator.predict(features_train_df)
test_risk_scores = estimator.predict(features_test_df)

median = np.median(train_risk_scores)
count_low = 0
count_high = 0
for index in range(0, len(test_risk_scores)):
    if test_risk_scores[index] > median:
        count_high += 1
        group.append(1)
    else:
        count_low += 1
        group.append(0)
print(count_low)
print(count_high)

0.5526611273298899
8
106


In [29]:
a = []
for index in range(0, len(event)):
    if event[index] == False:
        a.append(0)
    else:
        a.append(1)
print(*a, sep="; ")

0; 0; 0; 1; 1; 0; 0; 1; 1; 1; 1; 0; 1; 1; 1; 1; 0; 0; 0; 0; 1; 1; 1; 0; 1; 0; 0; 0; 0; 1; 1; 1; 1; 1; 1; 1; 0; 1; 0; 1; 1; 0; 0; 1; 0; 1; 1; 0; 1; 1; 1; 0; 1; 0; 1; 0; 1; 0; 0; 1; 1; 0; 1; 0; 1; 0; 1; 0; 0; 0; 1; 1; 1; 0; 1; 1; 1; 1; 1; 0; 0; 1; 1; 1; 1; 0; 1; 1; 1; 1; 0; 0; 0; 1; 0; 1; 0; 1; 0; 0; 0; 0; 1; 1; 0; 0; 0; 1; 1; 1; 1; 0; 1; 1


In [30]:
print(*survival_time, sep="; ")

1376; 1821; 1333; 1136; 3002; 2521; 3135; 23; 1400; 2014; 556; 2190; 2145; 779; 682; 450; 1278; 1326; 799; 346; 1790; 556; 534; 2889; 2522; 550; 1488; 1278; 1748; 132; 4; 611; 329; 982; 982; 522; 827; 898; 2400; 898; 306; 1088; 3889; 252; 2844; 765; 1432; 539; 80; 701; 376; 635; 30; 694; 369; 2816; 614; 2542; 2971; 2825; 424; 2847; 919; 1410; 7; 788; 872; 2292; 2092; 931; 1400; 498; 693; 2884; 2195; 746; 787; 1803; 150; 568; 681; 66; 429; 77; 661; 540; 702; 277; 2498; 737; 3364; 3297; 2162; 1726; 1156; 733; 2295; 662; 1725; 1249; 796; 645; 3002; 1525; 2770; 3364; 2521; 2014; 409; 2952; 1525; 763; 969; 337


In [32]:
print(*group, sep="; ")

1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 0; 0; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 0; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 0; 1; 0; 1; 1; 1; 1; 1; 1; 1; 1; 0; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 0; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 0; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1
